In [136]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
import numpy as np
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras.applications.vgg16 import VGG16
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, LearningRateScheduler, EarlyStopping

In [137]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###cargo los datos

In [159]:
X_train = np.load('/content/drive/MyDrive/Redes TPS/TP3/Data/X_train.npy')/255
Y_train = np.load('/content/drive/MyDrive/Redes TPS/TP3/Data/y_train.npy')
X_test = np.load('/content/drive/MyDrive/Redes TPS/TP3/Data/X_test.npy')/255

In [55]:
X_train[0,:,:,:].shape

(32, 32, 3)

In [160]:
Y_train = to_categorical(Y_train)

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

###Preprocesamiento

Aplicamos pixel centering, es decir, le restampo el promedio por canal a cada pixel. Este preprocesamiento es empleado por múltiples ganadores de competencias de clasificación de imágenes.

In [148]:
def pixel_centering(image):
  means = image.mean(axis=(0,1), dtype='float64')
  image -= means
  return image


In [149]:
for i in range(X_train.shape[0]):
  image = pixel_centering(X_train[0,:,:,:])
  X_train[0,:,:,:] = image

In [150]:
for i in range(X_test.shape[0]):
  image = pixel_centering(X_test[0,:,:,:])
  X_test[0,:,:,:] = image

###Creamos el modelo MobileNet

In [106]:
from keras.applications.vgg16 import preprocess_input
X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)


In [52]:
X_train.shape

(50000, 32, 32, 3)

In [151]:

base_model = VGG16(include_top=False, input_shape=(32,32,3), classes=100)

In [61]:
base_model.layers[10:25]

In [152]:

for layer in base_model.layers[:]:
    layer.trainable = False

In [93]:
#for layer in base_model.layers[16:20]:
#    layer.trainable = True

In [153]:
input_layer = base_model.layers[0].input
output_resnet = base_model.layers[-1].output
x = GlobalAveragePooling2D()(output_resnet)
x = Dense(100, activation="softmax")(x)

In [154]:
model_2 = Model(inputs=input_layer,outputs=x)

In [163]:
nb_train_samples = 50000
nb_validation_samples = 10000
epochs = 300
batch= 64

In [162]:
# create data generator
datagen = ImageDataGenerator(zoom_range=0.1, rotation_range = 5, horizontal_flip=True, shear_range = 0.1, validation_split=0.2)
# prepare iterator
it_train = datagen.flow(X_train, Y_train,shuffle=True, seed=42, batch_size=batch, subset='training')
it_valid = datagen.flow(X_train, Y_train,shuffle=True, seed=42, batch_size=batch, subset='validation')

In [164]:

lr = 0.001
loss = 'categorical_crossentropy'
optimizer = SGD()
metrics = ['Accuracy']

In [157]:
model_2.compile(optimizer=optimizer, loss=loss, metrics=metrics )

In [ ]:
checkpoint = ModelCheckpoint('/content/drive/MyDrive/Redes TPS/TP3/bestmodel_dag_1.hdf5', monitor='val_Accuracy', mode = 'max', verbose=1, save_best_only=True)
history = model_2.fit(X_train, Y_train,validation_split=0.2 ,batch_size=batch, epochs=epochs, callbacks=checkpoint)

Epoch 1/300
624/625 [============================>.] - ETA: 0s - loss: 3.3881 - Accuracy: 0.2320
Epoch 00001: val_Accuracy improved from -inf to 0.22390, saving model to /content/drive/MyDrive/Redes TPS/TP3/bestmodel_dag_1.hdf5
625/625 [==============================] - 21s 33ms/step - loss: 3.3881 - Accuracy: 0.2319 - val_loss: 3.4075 - val_Accuracy: 0.2239
Epoch 2/300
624/625 [============================>.] - ETA: 0s - loss: 3.3491 - Accuracy: 0.2359
Epoch 00002: val_Accuracy improved from 0.22390 to 0.22800, saving model to /content/drive/MyDrive/Redes TPS/TP3/bestmodel_dag_1.hdf5
625/625 [==============================] - 21s 33ms/step - loss: 3.3492 - Accuracy: 0.2359 - val_loss: 3.3742 - val_Accuracy: 0.2280
Epoch 3/300
624/625 [============================>.] - ETA: 0s - loss: 3.3143 - Accuracy: 0.2403
Epoch 00003: val_Accuracy improved from 0.22800 to 0.23330, saving model to /content/drive/MyDrive/Redes TPS/TP3/bestmodel_dag_1.hdf5
625/625 [==============================] - 2